In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
class node(object):
    
    def __init__(self,state,inbound=None,outbound=None):
        self.state=state
        self.inbound={}
        self.outbound={}
        
    def __str__(self):
        string="State: "+str(self.state)
        return string
    
    def addInbound(self,comingFrom,arcCost):
        self.inbound[comingFrom]=arcCost
        
    def addOutbound(self,goingTo,arcCost):
        self.outbound[goingTo]=arcCost


In [3]:
class layer(object):
    
    def __init__(self, layer_no):
        self.layer_no=layer_no
        self.nodes={}
        
    def __str__(self):
        string="Layer no. {}, containing {} nodes".format(self.layer_no,len(self.nodes))
        return string
    
    def addNode(self,node):
        self.nodes[node.state]=node
        
    def getNodes(self):
        return self.nodes

In [4]:
class arc(object):
    
    def __init__(self, head, tail, arctype, length):
        self.head=head
        self.tail=tail
        self.arctype=arctype
        self.length=length
    
    def __str__(self):
        string="Arc type: {}, Length: {}, Headnode: {}, Tailnode: {}".format(self.arctype,self.length,self.head.state, self.tail.state)
        return string

In [5]:
n=6
U=10

np.random.seed(78440)
    
cvals=np.random.randint(1,U,n)
wvals=np.random.randint(1,U,n)
W=sum(wvals)/2
d=[0,1]

In [6]:
n1=25
U1=50
    
cvals1=np.random.randint(1,U1,n1)
wvals1=np.random.randint(1,U1,n1)
W1=sum(wvals1)/2
d1=[0,1]

def read_instance(address = 'instance.csv'):
    data = pd.read_csv(address)
    c = data['c'].values
    w = data['w'].values
    W = data['W'].values[0]
    return c,w,W

In [7]:
def Algorithm_1(n,cvals,wvals,W,d):
    
    root_node=node(0)
    
    arcList=[]
    all_layers={}
    all_layers[0]={}
    all_layers[0][root_node.state]=root_node
    
    #Iterate through each layer
    for x in range(1,n):
        #Create next layer
        currLayer=all_layers[x-1]
        all_layers[x]={}
        nextLayer=all_layers[x]
        #Iterate through each node and action decision
        for currW, nodeInstance in currLayer.items():
            for decision in d:
                newstate=currW+(wvals[x-1]*decision)
                #Feasability check
                #Check if the new state already exists in the layer
                if newstate<=W:
                    if newstate in nextLayer.keys():
                        #Add arc into arclist
                        cost=cvals[x-1]*decision
                        #Add arc into node inbound and outbound
                        nodeInstance.outbound[nextLayer[newstate]]=cost
                        nextLayer[newstate].inbound[nodeInstance]=cost
                    else:
                        #Create new node, add arc to arclist
                        new_node=node(newstate)
                        nextLayer[newstate]=new_node
                        cost=cvals[x-1]*decision
                        #Add arc to node inbound and outbound
                        nodeInstance.outbound[new_node]=cost
                        new_node.inbound[nodeInstance]=cost
        
    #Changed from W because Rachel said so
    terminal_node=node(-999)
    all_layers[n]={}
    all_layers[n][-999]=terminal_node
    
    for currW, nodeInstance in all_layers[n-1].items():
        for decision in d:
            newstate=nodeInstance.state+(wvals[n-1]*decision)
            if newstate<=W:
                cost=cvals[n-1]*decision
                #new_arc=arc(nodeInstance,terminal_node,decision,cvals[n-1]*decision)
                #arcList.append(new_arc)
                nodeInstance.outbound[terminal_node]=cost
                terminal_node.inbound[nodeInstance]=cost
                
    final_node=node(0)            
    terminal_node.outbound[final_node]=0
                
    return all_layers, arcList

In [8]:
def Algorithm_2(all_layers,arcList,n):
    #Final reduction algorithm here we go
    reduced_layers=all_layers.copy()
    reduced_arcList=arcList.copy()
    
    two_arc=[]
    zero_arc=[]
    one_arc=[]
        
    #Second last layer reduction first, to save more time
    for nodeInstance in reduced_layers[n-1].values():
        if(len(nodeInstance.outbound.keys())==2):
            two_arc.append(nodeInstance)
        elif(list(nodeInstance.outbound.values())[0]>0):
            one_arc.append(nodeInstance)
        else:
            zero_arc.append(nodeInstance)
    
    #Two arcs to terminal
    for extraNode in two_arc[1:]:
        for comingFrom, arcCost in extraNode.inbound.items():
            comingFrom.outbound[two_arc[0]]=arcCost
            two_arc[0].inbound[comingFrom]=arcCost
            del(comingFrom.outbound[extraNode])
        del(reduced_layers[n-1][extraNode.state])
    
    #Take item arc to terminal
    for extraNode in one_arc[1:]:
        for comingFrom, arcCost in extraNode.inbound.items():
            comingFrom.outbound[one_arc[0]]=arcCost
            one_arc[0].inbound[comingFrom]=arcCost
            del(comingFrom.outbound[extraNode])
        del(reduced_layers[n-1][extraNode.state])
            
    #Dont take item to terminal arc        
    for extraNode in zero_arc[1:]:
        for comingFrom, arcCost in extraNode.inbound.items():
            comingFrom.outbound[zero_arc[0]]=arcCost
            zero_arc[0].inbound[comingFrom]=arcCost
            del(comingFrom.outbound[extraNode])
        del(reduced_layers[n-1][extraNode.state])
    
    #Starting from the third last layer moving up
    for x in range(n-2,0,-1):
        #Split nodes up into single path and double path nodes
        #This will hopefully save a lot of time in comparing node by node
        two_path={}
        one_path={}
        for nodeInstance in reduced_layers[x].values():
            if len(nodeInstance.outbound.keys())==1:
                one_path[nodeInstance.state]=nodeInstance
            else:
                two_path[nodeInstance.state]=nodeInstance
        
        #Start iterating through one path nodes
        for node_1 in list(one_path.values()):
            to_remove=[]
            for node_2 in list(one_path.values()):
                if(node_1 is not node_2):
                    if(node_1.outbound.items()==node_2.outbound.items()):
                        for comingFrom, arcCost in node_2.inbound.items():
                            comingFrom.outbound[node_1]=arcCost
                            node_1.inbound[comingFrom]=arcCost
                            if node_2 in comingFrom.outbound.keys():
                                del(comingFrom.outbound[node_2])
                        #THIS DOES NOT DEAL WITH DELETING ARCS FROM THE LOCAL TERMINAL NODE, MAY BE CRUCIAL
                        to_remove.append(node_2)
            for remove_node in to_remove:
                if remove_node.state in reduced_layers[x]:
                    del(reduced_layers[x][remove_node.state])
        
        for node_1 in list(two_path.values()):
            for node_2 in list(two_path.values()):
                if (node_1 is not node_2):
                    if(node_1.outbound.items() == node_2.outbound.items()):
                        for comingFrom, arcCost in node_2.inbound.items():
                            comingFrom.outbound[node_1]=arcCost
                            node_1.inbound[comingFrom]=arcCost
                            if node_2 in comingFrom.outbound.keys():
                                del(comingFrom.outbound[node_2])
                        #THIS DOES NOT DEAL WITH DELETING ARCS FROM THE LOCAL TERMINAL NODE, MAY BE CRUCIAL
                        to_remove.append(node_2)
            for remove_node in to_remove:
                if remove_node.state in reduced_layers[x]:
                    del(reduced_layers[x][remove_node.state])
        
        '''
        for node_1 in list(reduced_layers[x].values()):
            to_remove=[]
            for node_2 in list(reduced_layers[x].values()):
                #Not the same node yay
                if(node_1 is not node_2 and node_1 in reduced_layers[x].values() and node_2 in reduced_layers[x].values()):       
                #All node conditions clear, proceed to check if merge is possible
                #If outgoing arcs lead to the same/same set of nodes
                    if(node_1.outbound.keys() == node_2.outbound.keys()):
                        terminal_1=list(node_1.outbound.keys())
                        terminal_2=list(node_2.outbound.keys())

                    #1 terminal node
                        if len(terminal_1)==1:
                            if(terminal_1[0].inbound[node_1]==terminal_2[0].inbound[node_2]):
                            #Transfer incoming arcs
                                for comingFrom, arcCost in node_2.inbound.items():
                                    comingFrom.outbound[node_1]=arcCost
                                    node_1.inbound[comingFrom]=arcCost
                                    del(comingFrom.outbound[node_2])
                            #Delete outgoing arcs
                                del(terminal_2[0].inbound[node_2])
                            #Remove node
                                to_remove.append(node_2)

                    #2 terminal nodes
                        else:
                            if (terminal_1[0].inbound[node_1]==terminal_2[0].inbound[node_2] and terminal_1[1].inbound[node_1]==terminal_2[1].inbound[node_2]):
                                #Transfer incoming arcs
                                for comingFrom, arcCost in node_2.inbound.items():
                                    comingFrom.outbound[node_1]=arcCost
                                    node_1.inbound[comingFrom]=arcCost
                                    del(comingFrom.outbound[node_2])
                                #Delete outgoing arcs
                                del(terminal_2[0].inbound[node_2])
                                del(terminal_2[1].inbound[node_2])
                                #Remove node
                                to_remove.append(node_2)
                
            for remove_node in to_remove:
                del(reduced_layers[x][remove_node.state])
            reduced_layers.update()'''
            
                            
    return reduced_layers, reduced_arcList

In [9]:
[cvals2,wvals2,W2]=read_instance()
n2=len(cvals2)

In [10]:
start1=time.time()
[all_layers,arcList]=Algorithm_1(n2,cvals2,wvals2,W2,d)
end1=time.time()
[reduced_layers,reduced_arcList]=Algorithm_2(all_layers,arcList,n2)
end2=time.time()

In [11]:
#Just printing stuff, no biggie
    
print("Algo 1 Runtime: {}ms".format(1000*(end1-start1)))
print("Algo 2 Runtime: {}ms".format(1000*(end2-end1)))
print("Total Runtime: {}ms".format(1000*(end2-start1)))

for key, layer in all_layers.items():
    print("Layer no: {}".format(key))
    print("Nodes in layer: {}".format(len(layer.values())))
    '''for currWeight, nodeInsance in layer.items():
        print("Node: {}".format(currWeight))
        for goingTo, arcCost in nodeInstance.outbound.items():
            print("Arc cost: {}".format(arcCost))
    print("")'''

Algo 1 Runtime: 209.89322662353516ms
Algo 2 Runtime: 85518.01109313965ms
Total Runtime: 85727.90431976318ms
Layer no: 0
Nodes in layer: 1
Layer no: 1
Nodes in layer: 2
Layer no: 2
Nodes in layer: 4
Layer no: 3
Nodes in layer: 6
Layer no: 4
Nodes in layer: 12
Layer no: 5
Nodes in layer: 21
Layer no: 6
Nodes in layer: 38
Layer no: 7
Nodes in layer: 57
Layer no: 8
Nodes in layer: 76
Layer no: 9
Nodes in layer: 93
Layer no: 10
Nodes in layer: 114
Layer no: 11
Nodes in layer: 133
Layer no: 12
Nodes in layer: 150
Layer no: 13
Nodes in layer: 170
Layer no: 14
Nodes in layer: 178
Layer no: 15
Nodes in layer: 199
Layer no: 16
Nodes in layer: 205
Layer no: 17
Nodes in layer: 219
Layer no: 18
Nodes in layer: 230
Layer no: 19
Nodes in layer: 243
Layer no: 20
Nodes in layer: 265
Layer no: 21
Nodes in layer: 282
Layer no: 22
Nodes in layer: 295
Layer no: 23
Nodes in layer: 314
Layer no: 24
Nodes in layer: 323
Layer no: 25
Nodes in layer: 325
Layer no: 26
Nodes in layer: 337
Layer no: 27
Nodes in lay